In [254]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [255]:
liste = []
a=1

while a <= 80 :
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}
    r = requests.get("https://www.hepsiburada.com/ara?q=tablet&sayfa="+str(a)+"",headers=headers)
    soup = BeautifulSoup(r.content, "lxml")
    product_content_div = soup.find("div", attrs={"class": "productListContent-pXUkO4iHa51o_17CBibU"})
    product_list_ul = product_content_div.find("ul", attrs={"class":"productListContent-frGrtf5XrVXRwJ05HUfU productListContent-rEYj2_8SETJUeqNhyzSm"})
    product_items = product_list_ul.find_all("li", attrs={"class": "productListContent-zAP0Y5msy8OHn5z7T_K_"})

    for product_item in product_items:
        base_link = "https://www.hepsiburada.com/"
        link_end = product_item.a.get("href")
        link = base_link + link_end
       
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}
        r1 = requests.get(link, headers=headers)
        soup1 = BeautifulSoup(r1.content, "lxml")
       
        brand = soup1.find("a", attrs={"data-bind": "attr: {'data-hbus': userInformation() && userInformation().userId && isEventReady() ? productDetailHbus('BrandClick') : '' }"})
        brand = brand.text.strip() if brand else None
     
        price_element = soup1.find("del", attrs={"id": "originalPrice"})
        price = price_element.text.strip() if price_element else None
        
        properties = soup1.find("table", attrs={"class": "data-list tech-spec"})
        prop_soup = BeautifulSoup(str(properties), "html.parser")  
        prop_dict = {}

        if properties:
            prop_soup = BeautifulSoup(str(properties), "html.parser")
            for tr in prop_soup.find_all("tr"):
                th = tr.find("th")
                td = tr.find("span")
                
                prop_dict[prop_title] = prop_value
                          
        row_dict = {
            'marka': brand,
            'fiyat': price,
            'ekrancozunurluk': prop_dict.get('Max Ekran Çözünürlüğü',None),
            'kalem': prop_dict.get('Kalem Uyumluluğu',None),
            'ram': prop_dict.get('Ram Kapasitesi',None),
            'pil ':  prop_dict.get('Pil Gücü (mAh)',None),
            'isletimsistemitabani': prop_dict.get('İşletim Sistemi Tabanı',None),
            'isletimsistemi': prop_dict.get('İşletim Sistemi',None),
            'islemcimarka': prop_dict.get('İşlemci Markası',None),
            'islemcicekirdek': prop_dict.get('İşlemci Çekirdek Sayısı',None),
            'ekranboyutu': prop_dict.get('Ekran Boyutu',None),
            'HDMI': prop_dict.get('HDMI',None),
            'diskkapasitesi': prop_dict.get('Disk Kapasitesi',None),
            'MobilBağlantı': prop_dict.get('Mobil Bağlantı',None),
            'EkranModeli': prop_dict.get('Ekran Modeli',None),
            'Bluetooth': prop_dict.get('Bluetooth',None)
        }

        liste.append(row_dict)
     
    a += 1
     

In [256]:
df = pd.DataFrame(liste)
df.to_excel("hepsio.xlsx", index=False)